In [ ]:
import re
import pandas as pd

LOG_PATH = '../data/logfiles.log'
LOG_PATTERN = r'(?P<ip>\S+) - - \[(?P<time>[^\]]+)\] "(?P<method>\S+) (?P<endpoint>\S+) HTTP/\d\.\d" (?P<status>\d+) (?P<bytes>\d+) "(?P<referrer>[^"]+)" "(?P<ua>[^"]+)" (?P<response_time>\d+)'

def droptime(df):
    """
    Drop time
    
    It's all the same
    """
    df.drop(["time"], axis=1)

def parse_logs(path):
    data = []
    with open(path) as f:
        for line in f:
            match = re.match(LOG_PATTERN, line)
            if match:
                data.append(match.groupdict())
    
    return droptime(pd.DataFrame(data))

df = parse_logs(LOG_PATH)
df.head()


,ip,time,method,endpoint,status,bytes,referrer,ua,response_time
0,78.227.234.134,27/Dec/2037:12:00:00 +0100,GET,/usr,304,4994,-,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3)...,713
1,8.108.176.251,27/Dec/2037:12:00:00 +0100,DELETE,/usr,303,5035,-,Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000)...,1906
2,3.173.150.154,27/Dec/2037:12:00:00 +0100,GET,/usr/register,404,4983,http://www.garza.com/search/wp-content/blogabo...,Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000)...,3680
3,116.115.120.25,27/Dec/2037:12:00:00 +0100,PUT,/usr,200,4924,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2788
4,160.26.158.113,27/Dec/2037:12:00:00 +0100,PUT,/usr/admin,404,5043,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,793


In [20]:
df.drop(["time"], axis=1)

,ip,method,endpoint,status,bytes,referrer,ua,response_time
0,78.227.234.134,GET,/usr,304,4994,-,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3)...,713
1,8.108.176.251,DELETE,/usr,303,5035,-,Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000)...,1906
2,3.173.150.154,GET,/usr/register,404,4983,http://www.garza.com/search/wp-content/blogabo...,Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000)...,3680
3,116.115.120.25,PUT,/usr,200,4924,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2788
4,160.26.158.113,PUT,/usr/admin,404,5043,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,793
...,...,...,...,...,...,...,...,...
9995,176.113.252.35,GET,/usr/admin/developer,403,4995,http://www.garza.com/search/wp-content/blogabo...,Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000)...,2787
9996,81.8.116.146,POST,/usr/register,404,4983,http://www.garza.com/search/wp-content/blogabo...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3020
9997,99.46.31.81,DELETE,/usr/login,500,4906,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1004
9998,120.158.148.193,DELETE,/usr/admin/developer,403,4968,http://www.garza.com/search/wp-content/blogabo...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2540


In [18]:
df["time"].unique()

array(['27/Dec/2037:12:00:00 +0100'], dtype=object)